# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Project Submission

### Data Initialization

In [1]:
import os

import pandas as pd

from data_management import DataManager

DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
aisles = pd.read_csv(os.path.join(DATA_DIR, 'aisles.csv.zip'))
departments = pd.read_csv(os.path.join(DATA_DIR, 'departments.csv.zip'))
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv.zip'))

data_manager = DataManager(op_prior, op_train, tip_train, tip_test, orders, products, aisles, departments)
order_amount = len(data_manager.get_orders_tip())

### Feature Engineering

In [ ]:
from feature_engineering.static_features import TipHistory, ReorderedRatio, OrderSize, PrevTippedProductsRatio, \
    CustomerLifetime, PrevOrderTipped, OrderFrequency, SimOrdersTipRatio, AvgSizePrevOrders, MeanOrderedRate, \
    LastTipSequence, RelDaysSinceTip, DaysSinceTip, OrderNumberSquared, HodHighTipProbability, DowHighTipProbability, \
    ContainsAlcohol

from feature_engineering.dynamic_features import ProductTipRate, DepartmentTipRate, AisleTipRate, DynamicFeatureTest1, \
    DynamicFeatureTest2, AssocRulesAisles, AssocRulesDepartments

# Kilian
tip_history = TipHistory()
reordered_rate = ReorderedRatio()
order_size = OrderSize()
prev_tipped_products_ratio = PrevTippedProductsRatio()
customer_lifetime = CustomerLifetime()
prev_order_tipped = PrevOrderTipped()

# Daniel
order_frequency = OrderFrequency()
mean_ordered_rate = MeanOrderedRate()
rel_days_since_tip = RelDaysSinceTip()
days_since_tip = DaysSinceTip()

# Max
sim_orders_tip_ratio = SimOrdersTipRatio()
product_tip_rate = ProductTipRate()
department_tip_rate = DepartmentTipRate()
aisle_tip_rate = AisleTipRate()
last_tip_sequence = LastTipSequence()

# Tom
avg_size_prev_orders = AvgSizePrevOrders()

# Features analysis
order_number_squared = OrderNumberSquared()
hod_high_tip_probability = HodHighTipProbability()
dow_high_tip_probability = DowHighTipProbability()
contains_alcohol = ContainsAlcohol()

assoc_rules_departments = AssocRulesDepartments()
assoc_rules_aisles = AssocRulesAisles()

# Test
dynamic_feature_test_1 = DynamicFeatureTest1()
dynamic_feature_test_2 = DynamicFeatureTest2()

In [ ]:
# Static Features
data_manager.register_feature(tip_history)
data_manager.register_feature(reordered_rate)
data_manager.register_feature(order_size)
data_manager.register_feature(customer_lifetime)
data_manager.register_feature(prev_order_tipped)
data_manager.register_feature(prev_tipped_products_ratio)

data_manager.register_feature(order_frequency)
data_manager.register_feature(sim_orders_tip_ratio)
data_manager.register_feature(avg_size_prev_orders)
data_manager.register_feature(mean_ordered_rate)
data_manager.register_feature(last_tip_sequence)
data_manager.register_feature(rel_days_since_tip)
data_manager.register_feature(days_since_tip)

data_manager.register_feature(order_number_squared)
data_manager.register_feature(hod_high_tip_probability)
data_manager.register_feature(dow_high_tip_probability)
data_manager.register_feature(contains_alcohol)

# Dynamic Features
data_manager.register_feature(dynamic_feature_test_1)
data_manager.register_feature(dynamic_feature_test_2)
data_manager.register_feature(product_tip_rate)
data_manager.register_feature(department_tip_rate)
data_manager.register_feature(aisle_tip_rate)

data_manager.register_feature(assoc_rules_departments)
data_manager.register_feature(assoc_rules_aisles)

In [ ]:
data_manager.compute_features()

In [ ]:
data_manager.get_orders_tip().head(100)

In [ ]:
data_manager.export_features('data/prepared_data/computed_features.csv.zip', only_static=False)

In [ ]:
data_manager.import_features('data/prepared_data/computed_features.csv.zip', only_static=False)

In [ ]:
# data_manager.get_orders_tip().head(100)

In [ ]:
# test = data_manager.get_orders_tip()

In [ ]:
# print(f"Number of orders: {len(data_manager.get_orders_tip())}")
# print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
# print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

In [ ]:
# order_ids = data_manager.get_orders_tip(complete=True).groupby('user_id')['order_id'].head(5)
# order_amount = len(order_ids)

In [ ]:
# data_manager.set_subset(order_ids)

In [ ]:
# data_manager.get_orders_tip().head(25)

In [ ]:
# print(f"Number of orders: {len(data_manager.get_orders_tip())}")
# print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
# print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

In [ ]:
# order_ids = data_manager.get_orders_tip(complete=True).groupby('user_id')['order_id'].head(4)
# order_amount = len(order_ids)
# 
# data_manager.set_subset(order_ids)
# data_manager.get_orders_tip().head(25)

In [ ]:
# print(f"Number of orders: {len(data_manager.get_orders_tip())}")
# print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
# print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

### Analysis
- Only orders from the training set (orders with tip information) are considered

In [ ]:
# from analysis import DayOfWeek, HourOfDay, DaysSincePriorOrder, OrderNumber, Department, Aisle, Product, GeneralAnalysis

In [ ]:
# day_of_week = DayOfWeek(data_manager)
# hour_of_day = HourOfDay(data_manager)
# days_since_prior_order = DaysSincePriorOrder(data_manager)
# order_number = OrderNumber(data_manager)
# department = Department(data_manager)
# aisle = Aisle(data_manager)
# product = Product(data_manager)
# number_order_user = NumberOrderUser(data_manager)
# general_analysis = GeneralAnalysis(data_manager)

In [ ]:
# day_of_week.execute_analysis()

In [ ]:
# hour_of_day.execute_analysis()

In [ ]:
# days_since_prior_order.execute_analysis()

In [ ]:
# order_number.execute_analysis()

In [ ]:
# department.execute_analysis()

In [ ]:
# aisle.execute_analysis()

In [ ]:
# product.execute_analysis()

In [ ]:
# number_order_user.execute_analysis()

### Data Preparation

In [27]:
# from feature_engineering.static_features import DowHighTipProbability, HodHighTipProbability, OrderNumberSquared, \
#     ContainsAlcohol
#
# data_manager.register_feature(DowHighTipProbability())
# data_manager.register_feature(HodHighTipProbability())
# data_manager.register_feature(OrderNumberSquared())
# data_manager.register_feature(ContainsAlcohol())

In [28]:
# data_manager.compute_features()

### Model Training & Evaluation

Order is important:
- Remove first order of each user and assign the dataset for parameter tuning
- Initialize cross-validation splitter
- Export splits
- Tune the parameters of the model with the dataset from 1.

In [29]:
from data_management.cross_validation import LastOrderUserTSCVSplitter

In [30]:
orders_tip = data_manager.get_orders_tip()
orders_tip.shape

(3346083, 12)

In [31]:
# order_ids = orders_tip[orders_tip['order_number'] > 1]['order_id']
# order_ids = orders_tip['order_id']
# data_manager.set_subset(order_ids)
data_manager.remove_first_orders()

In [32]:
orders_tip_train = data_manager.get_orders_tip_train()
orders_tip_train.shape

(3008665, 12)

In [33]:
# cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [34]:
last_order_user_tscv_splitter = LastOrderUserTSCVSplitter(data_manager, n_splits=5)

In [35]:
splits = last_order_user_tscv_splitter.split(orders_tip_train)

In [36]:
folds = {}
prev_test = 0
for i, (train_index, test_index) in enumerate(splits):
    print(f'Fold {i + 1}')
    print(f'Train: {len(train_index)}')
    print(f'Test: {len(test_index)}')
    print(f'Orders: {len(orders_tip_train)}')
    print(f'Sum equal: {(len(train_index) + len(test_index) + prev_test) == len(orders_tip_train)}')
    print(f'Ratio: {len(test_index) / (len(train_index) + len(test_index))}')

    train_orders = orders_tip_train.loc[train_index]
    test_orders = orders_tip_train.loc[test_index]
    current_fold = {
        'train': train_orders,
        'test': test_orders
    }
    folds[f'fold_{i}'] = current_fold
    prev_test += len(test_index)

In [37]:
last_order_user_tscv_splitter.export_splits('data/prepared_data/')

Export 1: Train size: 2802456, Test size: 206209
Export 2: Train size: 2596247, Test size: 206209
Export 3: Train size: 2414024, Test size: 182223
Export 4: Train size: 2251391, Test size: 162633
Export 5: Train size: 2104923, Test size: 146468


In [38]:
# user_tscv_splitter = UserTSCVSplitter(data_manager, n_splits=5, validation_set_ratio=0.2, seed=42)

In [39]:
# splits = user_tscv_splitter.split(orders_tip_train)

In [40]:
# folds = {}
# prev_test = 0
# for i, (train_index, test_index) in enumerate(splits):
#     print(f'Fold {i + 1}')
#     print(f'Train: {len(train_index)}')
#     print(f'Test: {len(test_index)}')
#     print(f'Orders: {len(orders_tip_train)}')
#     print(f'Sum equal: {(len(train_index) + len(test_index) + prev_test) == len(orders_tip_train)}')
#     print(f'Ratio: {len(test_index) / (len(train_index) + len(test_index))}')
# 
#     train_orders = orders_tip_train.loc[train_index]
#     test_orders = orders_tip_train.loc[test_index]
#     current_fold = {
#         'train': train_orders,
#         'test': test_orders
#     }
#     folds[f'fold_{i}'] = current_fold
#     prev_test += len(test_index)

In [41]:
# user_tscv_splitter.export_splits('data/prepared_data/')

In [42]:
# features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_size', 'tip_history',
#             'dynamic_feature_test_1', 'dynamic_feature_test_2']
#
# prepared_splits_dict = user_tscv_splitter.import_splits('data/prepared_data/', features)

In [43]:
features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_size', 'tip_history',
            'dynamic_feature_test_1', 'dynamic_feature_test_2']

prepared_splits_dict = last_order_user_tscv_splitter.import_splits('data/prepared_data/', features)

In [44]:
from data_management import DatasetSelector
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from tempfile import mkdtemp
from joblib import Memory

# TODO: Add scaling & feature selection
pipeline_steps = [('selector', DatasetSelector(prepared_splits_dict)),
                  ('estimator', DecisionTreeClassifier())]

cachedir = mkdtemp()
memory = Memory(location=cachedir, verbose=0)

pipeline = Pipeline(pipeline_steps, verbose=False, memory=memory)

In [45]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'estimator__criterion': ['entropy'],
    'estimator__max_depth': [9, 10],
    'estimator__min_samples_leaf': [256]
}

grid_search_clf = GridSearchCV(pipeline, param_grid, cv=last_order_user_tscv_splitter, scoring='accuracy', verbose=1,
                               n_jobs=-1)

In [46]:
X = orders_tip_train[features]
y = orders_tip_train['tip'].astype(bool)

In [47]:
X.head(25)

,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_size,tip_history,dynamic_feature_test_1,dynamic_feature_test_2
0,2,3,7,15.0,6,0.000000,0.100000,0.100000
1,3,3,12,21.0,5,0.000000,0.200000,0.200000
2,4,4,7,29.0,5,0.000000,0.300000,0.300000
3,5,4,15,28.0,8,0.000000,0.400000,0.400000
4,6,2,7,19.0,4,0.000000,0.500000,0.500000
5,7,1,9,20.0,5,0.000000,0.600000,0.600000
6,8,1,14,14.0,6,0.000000,0.700000,0.700000
7,9,1,16,0.0,6,0.000000,0.800000,0.800000
8,10,4,8,30.0,9,0.000000,0.900000,0.900000
9,2,5,10,10.0,6,0.000000,0.071429,0.071429


In [ ]:
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

In [ ]:
grid_search_clf.fit(X, y)

In [ ]:
cv_results = grid_search_clf.cv_results_
result_list = [grid_search_clf.cv_results_[f'split{i}_test_score'][grid_search_clf.best_index_] for i in range(5)]

for i, result in enumerate(result_list):
    print(f'Accuracy (Fold {i + 1}): {result}')

print(f'\nMean Accuracy: {cv_results["mean_test_score"][grid_search_clf.best_index_]}')

In [ ]:
grid_search_clf.best_score_

In [ ]:
grid_search_clf.best_params_

In [ ]:
best_estimator = grid_search_clf.best_estimator_

### Validation

In [ ]:
# features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_size', 'tip_history',
#             'dynamic_feature_test_1', 'dynamic_feature_test_2']
#
# prepared_splits_dict = last_order_user_tscv_splitter.import_splits('data/prepared_data/', features)

In [ ]:
# all_orders = data_manager.get_orders_tip(complete=True)
# all_orders.shape

In [ ]:
# data_manager.set_subset(all_orders['order_id'])

In [ ]:
# validation_test = data_manager.get_orders_tip_test()
# validation_test.shape

In [ ]:
# validation_train = data_manager.get_orders_tip_train()
# validation_train.shape

In [ ]:
# validation_train_without_first = validation_train[validation_train['order_number'] > 1]
# validation_train_without_first.shape

In [ ]:
# prepared_splits_dict[-1147446606210854265].shape[0]

In [ ]:
# all_orders_without_first = all_orders[all_orders['order_number'] > 1].reset_index(drop=True)
# all_orders_without_first.shape

In [ ]:
# validation_train_without_first[features].reset_index(drop=True)

In [ ]:
# prepared_splits_dict[-1147446606210854265]

In [ ]:
# round(prepared_splits_dict[-1147446606210854265], 6).equals(round(validation_train_without_first[
#                                                                       features].reset_index(drop=True), 6))

In [ ]:
# validation_test[features]

In [ ]:
# prepared_splits_dict[4027820417624348460]

In [ ]:
# round(prepared_splits_dict[4027820417624348460].reset_index(drop=True), 6).equals(round(
#     validation_test[features].reset_index(drop=True), 6))

### Prediction

IMPORTANT:
- Before predicting, the indices must probably be reseted, for X and y
- To save the results, we must join the predictions with the template based on the ORDER_ID, beause the indices don't align


In [ ]:
# orders_tip_test = data_manager.get_orders_tip_test()
# orders_tip_test['tip'] = best_estimator.predict(orders_tip_test[features])

In [ ]:
# orders_tip_test_csv = orders_tip_test[tip_test.columns].copy()
# orders_tip_test_csv.rename(columns={tip_test.columns[0]: ''}, inplace=True)
# orders_tip_test_csv

In [ ]:
# orders_tip_test_csv.to_csv(os.path.join(DATA_DIR, 'tip_testdaten1.csv'), index=False)